# Pre-processing of visual input

Images are pre-processed through the application of the histogram of oriented gradients (HOG) algorithm.

The Pre-processing algorithm is applied to the images to make them suitable to be encoded into the neural circuits. It is inspired by the computations that take place in the first layers of the visual system of the primates,but it is not a realistic description of them.

1. The HOG filter is applied (14x14 cell size, 14x14 block size, 9 orientation bins and 7x7 block stride).

2. The size of original images is 28 × 28 pixel.

3. Histograms are computed using cells of 14 × 14 size that are applied on the images using a striding step of 7 pixels, resulting in 9 histograms per image.

4. Each cell provides an histogram with 9 bins, each bin assuming a real value.

5. 4 values binning: Each bin is then transformed into a vector of 4 mutually exclusive truth values. The coding of each input feature from acontinuous domain of possible intensities, e.g from the (0.0,1.0) range of possible values,into the discrete activity of subsets of individual thalamic neurons should preserve, at least partially, a distance-like notion (UltraLow:[0; 0.25), MediumLow:[0.25; 0.50), MediumHigh:[0.50; 0.75), High:[0.75,1.0]).

In summary, each image has been transformed in a vector of 324 binary features.

The resulting visual input is presented to the network through the thalamus, where each feature provides a binary input to a different thalamic cell.

HOG documentation: https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.hog

In [32]:
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt
import tensorflow as tf

In [33]:
mnist = tf.keras.datasets.mnist # 28 x 28 pixels
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [40]:
# sample_image = x_train[0]

## HOG transformation

In [48]:
# Corrected function for HOG transformation with specific stride
def hog_transformation(image, stride=7):
    feature_vectors = []

    # Size of the image, cells, and blocks
    height, width = image.shape
    cell_size = block_size = 14

    # Iterate over the image with the given stride
    for y in range(0, height - cell_size + 1, stride):
        for x in range(0, width - cell_size + 1, stride):
            cell_region = image[y:y+cell_size, x:x+cell_size]

            # Calculate HOG features for the cell
            fd, hog_image = hog(
                cell_region,
                orientations=9,
                pixels_per_cell=(cell_size, cell_size),
                cells_per_block=(1, 1),
                visualize=True,
                feature_vector=True
            )

            feature_vectors.append(fd)

    # Concatenate all feature vectors
    feature_vector = np.concatenate(feature_vectors)

    # print("Feature vector:", feature_vector)
    # print("Feature vector length:", len(feature_vector))
    # print("Feature descriptor:", fd)
    # print("Feature descriptor length:", len(fd))

    return feature_vector, hog_image


(array([0.41049011, 0.        , 0.19546443, 0.41789258, 0.41789258,
        0.21716949, 0.39357862, 0.41789258, 0.2594652 , 0.38777022,
        0.22175656, 0.28084969, 0.38777022, 0.38777022, 0.37856621,
        0.35460294, 0.38777022, 0.03780066, 0.36014781, 0.30524331,
        0.4210823 , 0.4210823 , 0.4210823 , 0.41133185, 0.25572006,
        0.        , 0.1029685 , 0.43937762, 0.        , 0.18784964,
        0.08509439, 0.43937762, 0.26329971, 0.43937762, 0.43937762,
        0.34049073, 0.37769285, 0.27588264, 0.22756847, 0.37769285,
        0.37769285, 0.37769285, 0.37769285, 0.37769285, 0.12723999,
        0.40453902, 0.31073204, 0.        , 0.40453902, 0.40453902,
        0.40453902, 0.40453902, 0.        , 0.29186725, 0.1338323 ,
        0.10788827, 0.56087562, 0.56087562, 0.56087562, 0.        ,
        0.10235298, 0.        , 0.12739124, 0.43866571, 0.08929545,
        0.43866571, 0.43866571, 0.43866571, 0.14356505, 0.43866571,
        0.        , 0.09631918, 0.39509041, 0.16

In [35]:
def bin_feature_values(feature_vector):
    '''
    Binning the feature values into four categories
    '''

    binned_vector = []

    for value in feature_vector:
        if value < 0.25:
            binned_vector.append([1, 0, 0, 0])  # UltraLow
        elif value < 0.50:
            binned_vector.append([0, 1, 0, 0])  # MediumLow
        elif value < 0.75:
            binned_vector.append([0, 0, 1, 0])  # MediumHigh
        else:
            binned_vector.append([0, 0, 0, 1])  # High

    return np.array(binned_vector).flatten()    # flattening preserves properties but in 1-dimension array


In [36]:
# Function to plot the original image and its HOG representation side by side
def plot_images(original_image, hog_image):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(original_image, cmap='gray')
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    plt.imshow(hog_image, cmap='gray')
    plt.title('HOG Image')

    plt.show()


In [ ]:
def process_and_store(x):
  '''
  Process and store binned features vectors for training data
  '''

  binned_features = []

  for image in x:
      feature_vector, hog_image = hog_transformation(image) # Apply HOG transformation to each image

      binned_feature = bin_feature_values(feature_vector)   # Bin the feature vector values

      binned_features.append(binned_feature)                # Store the binned feature vector

      # plot_images(image, hog_image)                       # Plot original and HOG transformed images

  binned_features = np.array(binned_features)               # Convert list to numpy array

  return binned_features

feature_x_train = process_and_store(x_train).copy()
feature_x_test = process_and_store(x_test).copy()

# print(feature_x_train)
# print(feature_x_test)

# plt.show()


# Machine Learning Model

## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

## Model implementation

In [39]:
# Build a simple CNN model
input_shape = (324,)

model = models.Sequential([                 # Not original model
    layers.Input(shape=input_shape),
    layers.Dense(128, activation='relu'),   # You can adjust the number of neurons
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes for MNIST
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Incremental training
batch_size = 25
steps = len(feature_x_train) // batch_size

for step in range(steps):
    start = step * batch_size
    end = start + batch_size
    x_batch, y_batch = feature_x_train[start:end], y_train[start:end]

    # Train the model
    model.fit(x_batch, y_batch, epochs=1, verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(feature_x_test, y_test, verbose=0)
    print(f"After training step {step + 1}, accuracy: {accuracy:.4f}")

# 2400 steps, first trial: 'After training step 2400, accuracy: 0.8874'

After training step 1, accuracy: 0.1240
After training step 2, accuracy: 0.1532
After training step 3, accuracy: 0.1864
After training step 4, accuracy: 0.2123
After training step 5, accuracy: 0.2386
After training step 6, accuracy: 0.2585
After training step 7, accuracy: 0.2745
After training step 8, accuracy: 0.2909
After training step 9, accuracy: 0.3104
After training step 10, accuracy: 0.3288
After training step 11, accuracy: 0.3502
After training step 12, accuracy: 0.3717
After training step 13, accuracy: 0.3927
After training step 14, accuracy: 0.4104
After training step 15, accuracy: 0.4271
After training step 16, accuracy: 0.4363
After training step 17, accuracy: 0.4405
After training step 18, accuracy: 0.4471
After training step 19, accuracy: 0.4557
After training step 20, accuracy: 0.4666
After training step 21, accuracy: 0.4729
After training step 22, accuracy: 0.4834
After training step 23, accuracy: 0.4951
After training step 24, accuracy: 0.5111
After training step 25, a